# Лабораторная работа №2
## Симплекс метод для решения задач линейного программирования 
## Дополнительное задание

## Выполнил: Яковлев Артур, 853501
## Проверила: Костюкова О.И.

### Вариант 7

Импорт необходимых модулей

In [94]:
import numpy as np
import scipy.linalg as sla
from copy import deepcopy

Исходные данные для 7 варианта задания

In [116]:
dataset = {
    'A': np.array([
        [ 1., -1.,  0.,  2., -1., -1.,  2.,  1.,  7.],
        [ 1., -2., -1.,  1.,  0., -1.,  1.,  1., -3.],
        [-1., -1., -1.,  1.,  1., -1., -3., -1.,  4.],
        [-1., -3.,  0., -6.,  2.,  3.,  0.,  1.,  1.],
    ]),
    'c': np.array([-4., -1., -1., -3., -1., -4., -1., -4., -1.]),
    'b': np.array([113., 3., 21., -30.]),
    'lower': np.array([5., -5., -2., 4., -1., 0., -1., 1., 0.]),
    'upper': np.array([9., -1., 3., 9., 5., 4., 5., 6., 10.]),
    'Jb': [1, 2, 4, 5],
    'x': np.array([9., -3., 0., 9., 1., 2., 5., 6., 10.]),
}

eps = 1e-9

Проверим ранг матрицы $A$

In [48]:
np.linalg.matrix_rank(dataset['A'])

4

Далее реализуем выполнение итерации по шагам

In [117]:
def iteration(dataset, verbose=True):
    eps = 1e-9
    
    # Step 1
    B = sla.inv(dataset['A'][:, dataset['Jb']])
    u = dataset['c'][dataset['Jb']] @ B
    Jn = [i for i in range(dataset['A'].shape[1]) if i not in dataset['Jb']]
    delta_j = u @ dataset['A'] - dataset['c']
    
    if verbose:
        print(f'A_b = {dataset["A"][:, dataset["Jb"]]}')
        print(f'c_b = {dataset["c"][dataset["Jb"]]}')
        print(f'B = {B}')
        print(f'x = {dataset["x"]}')
        print(f'Jb = {dataset["Jb"]}')
    
        print('\nStep 1\n')
        print(f'u = {u}')
        print(f'delta j = {delta_j}')
        
    # Step 2
    j0 = None
    stop_iter = False
    for i in Jn:
        if np.abs(dataset['x'][i] - dataset['lower'][i]) < eps and delta_j[i] < -eps:
            j0 = i
            z0 = 1
            break
        if np.abs(dataset['x'][i] - dataset['upper'][i]) < eps and delta_j[i] > eps:
            j0 = i
            z0 = -1
            break
    
    if j0 is None:
        return dataset["x"]
    
    if verbose:
        print('\nStep 2\n')
        print(f'j0 = {j0}')
        print(f'z0 = {z0}')
    
    # Step 3
    z = np.concatenate((-z0 * B @ dataset['A'][:, j0], np.array([z0])), axis=0)
    
    if verbose:
        print('\nStep 3\n')
        print(f'z = {z}')
    
    # Step 4
    dataset['Jb'].append(j0)
    thetas = np.zeros(len(dataset['Jb']))
    for i in range(len(dataset['Jb'])):
        j_i = dataset['Jb'][i]
        if np.abs(z[i]) < eps:
            thetas[i] = np.inf
        elif z[i] > 0.:
            thetas[i] = (dataset['upper'][j_i] - dataset['x'][j_i]) / z[i]
        else:
            thetas[i] = (dataset['lower'][j_i] - dataset['x'][j_i]) / z[i]

    
    s = np.argmin(thetas)
    theta0 = thetas[s]
    
    if verbose:
        print('\nStep 4\n')
        print(f'thetas = {thetas}')
        print(f'theta0 = {theta0}')
        print(f's = {s}')
    
    # Step 5
    dataset['x'][dataset['Jb']] += theta0 * z
    dataset['Jb'].pop(-1)
    if len(dataset['Jb']) != s:
        dataset['Jb'].pop(s)
        dataset['Jb'].append(j0)
        dataset['Jb'].sort()
    
    if verbose:
        print('\nStep 5\n')
        print(f'New x = {dataset["x"]}')
        print(f'New Jb = {dataset["Jb"]}')


Реализация выполнения алгоритма

In [118]:
def solve(dataset, max_iter=20, verbose=True):
    copied_dataset = deepcopy(dataset)
    
    if verbose:
        print('Initial values')
        print(f'A = {dataset["A"]}')
        print(f'c = {dataset["c"]}')
        print(f'b = {dataset["b"]}')
        print(f'lower bound = {dataset["lower"]}')
        print(f'upper bound = {dataset["upper"]}')
        print(f'initial x = {dataset["x"]}')
        print(f'initial Jb = {dataset["Jb"]}')
    
    for i in range(max_iter):
        if verbose:
            print('\n===============================\n')
            print(f'Iteration {i + 1}')
        final_plan = iteration(copied_dataset, verbose=verbose)
        if final_plan is not None:
            print(f'\n\nBest plan: {final_plan}')
            return
    
    print(f'Best found plan: {copied_dataset["x"]}')
    

Запустим алгоритм на наших данных

In [119]:
solve(dataset)

Initial values
A = [[ 1. -1.  0.  2. -1. -1.  2.  1.  7.]
 [ 1. -2. -1.  1.  0. -1.  1.  1. -3.]
 [-1. -1. -1.  1.  1. -1. -3. -1.  4.]
 [-1. -3.  0. -6.  2.  3.  0.  1.  1.]]
c = [-4. -1. -1. -3. -1. -4. -1. -4. -1.]
b = [113.   3.  21. -30.]
lower bound = [ 5. -5. -2.  4. -1.  0. -1.  1.  0.]
upper bound = [ 9. -1.  3.  9.  5.  4.  5.  6. 10.]
initial x = [ 9. -3.  0.  9.  1.  2.  5.  6. 10.]
initial Jb = [1, 2, 4, 5]


Iteration 1
A_b = [[-1.  0. -1. -1.]
 [-2. -1.  0. -1.]
 [-1. -1.  1. -1.]
 [-3.  0.  2.  3.]]
c_b = [-1. -1. -1. -4.]
B = [[-0.6  0.2 -0.2 -0.2]
 [ 2.2 -2.4  1.4  0.4]
 [ 0.6 -1.2  1.2  0.2]
 [-1.   1.  -1.   0. ]]
x = [ 9. -3.  0.  9.  1.  2.  5.  6. 10.]
Jb = [1, 2, 4, 5]

Step 1

u = [ 1.8 -0.6  1.6 -0.4]
delta j = [ 4.00000000e+00 -8.88178420e-16  0.00000000e+00  1.00000000e+01
 -2.22044605e-16  0.00000000e+00 -8.00000000e-01  3.20000000e+00
  2.14000000e+01]

Step 2

j0 = 0
z0 = -1

Step 3

z = [-1.11022302e-16 -2.00000000e+00 -2.00000000e+00  1.00000000e+00
 -1